# Introduction

Steps necessary to build a RAG based conversational model:
1. Load documents
2. Split documents
3. Store documents
4. Retrieve documents
5. Generate answers based on retrieved documents

# Installation

## Necessary packages

To install necessary packages, run:

```sh
pip install openai==0.27.8
pip install tiktoken
pip install langchain
pip install langchainhub
pip install python-dotenv
pip install chromadb
```

# Code

## Necessary imports

In [1]:
import sys

import dotenv

from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma

from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [2]:
# Load OPENAI_API_KEY environment variable in .env file
dotenv.load_dotenv()

True

## Creating embeddings and chat models

In [3]:
# Constants for model names
CHAT_MODEL_NAME = 'gpt-3.5-turbo'
EMBEDDINGS_MODEL_NAME = 'text-embedding-ada-002'

In [4]:
chat = ChatOpenAI(model_name=CHAT_MODEL_NAME, temperature=0)

In [5]:
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL_NAME)

## Loading Comvest data, processing and storing it

In [6]:
loader = AsyncHtmlLoader(["https://www.pg.unicamp.br/norma/31594/0"])
html2text = Html2TextTransformer()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap = 10
)

In [8]:
docs = loader.load()

Fetching pages: 100%|###########################################################################################################| 1/1 [00:00<00:00,  3.42it/s]


In [9]:
docs_transformed = html2text.transform_documents(docs)

In [10]:
docs_splits = text_splitter.split_documents(docs_transformed)

In [11]:
vectordb = Chroma.from_documents(
    documents=docs_splits,
    embedding=embeddings,
    persist_directory='./vectorstore'
)
vectordb.persist()

## Prompt Template

In [12]:
SYSTEM_MESSAGE_TEMPLATE = """Considere a conversa, o contexto e a pergunta dada para dar uma resposta. Caso você não saiba uma resposta, fale 'Me desculpe, mas não tenho uma resposta para esta pergunta' em vez de tentar inventar algo.
----
Conversa:
{chat_history}
----
Contexto:
{context}
----
"""

HUMAN_MESSAGE_TEMPLATE = """
Pergunta:
{question}
"""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(SYSTEM_MESSAGE_TEMPLATE),
    HumanMessagePromptTemplate.from_template(HUMAN_MESSAGE_TEMPLATE),
])

## Augmentation function

In [13]:
def augment_prompt(question, chat_history, retriever, prompt_template):
    docs = retriever.get_relevant_documents(query=question)
    
    docs_formatted = '\n'.join([doc.page_content for doc in docs])
    chat_history_formatted = '\n'.join([f"Usuário: {exchange[0]}\nSistema: {exchange[1]}\n" for exchange in chat_history])
    
    prompt = prompt_template.format_messages(
        input_language="Portuguese", 
        output_language="Portuguese", 
        question=question,
        context=docs_formatted,
        chat_history=chat_history_formatted
    )
    return prompt

## Defining document retrievers

In [14]:
db_retriever = vectordb.as_retriever()

In [15]:
RETRIEVER_PROMPT_TEMPLATE = """Você é um assistente de modelo de linguagem de IA. 
Sua tarefa é gerar 3 versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de um banco de dados vetorial.
Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar o usuário a superar algumas das limitações da pesquisa de similaridade baseada em distância. 
Forneça estas perguntas alternativas separadas por novas linhas. 
Pergunta original: {question}
"""

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_type="mmr"), 
    llm=chat,
    prompt=PromptTemplate(
        input_variables=['question'], 
        template=RETRIEVER_PROMPT_TEMPLATE
    )
)

In [16]:
question = "Como é o processo de inscrição pelo ENEM na Unicamp?"

docs = retriever_from_llm.get_relevant_documents(query=question)

## ChatBot demonstration

In [17]:
prompt = augment_prompt(
    question="Como é o processo de inscrição pelo ENEM na Unicamp?", 
    chat_history=[],
    retriever=db_retriever, 
    prompt_template=prompt_template,
)

In [18]:
result = chat(prompt)

In [21]:
result

AIMessage(content='O processo de inscrição pelo ENEM na Unicamp é realizado através da Comvest, que é responsável pela seleção de candidatos. As instruções necessárias para a inscrição, o Manual do Ingresso e as informações sobre a Unicamp e seus cursos estão disponíveis na página da Comvest (www.comvest.unicamp.br). Os candidatos isentos da Taxa de Inscrição serão dispensados do recolhimento dessa taxa. O processo de inscrição somente será validado com o recolhimento da Taxa de Inscrição. A situação da inscrição deverá ser consultada pelo candidato na página da Comvest a partir de 72 horas após a inscrição.')